# Exchange Trade Protocol

In investory must consider factors such as downside risk, market conditions, and the length of time it will take for each investment to realize returns. They also need to consider opportunity costs that represent the potential benefits that an individual, investor, or business misses out on when choosing the ETP over another.

The _Return On Investment_ (ROI) of a single investment is the net price gain from holding the asset by the asset's original cost. The cost of an asset includes not only the purchase price, but also any commissions, management fees, or other expenses associated with the acquisition. __ETP ROI__ is the differential gain (or loss) of the sum of the weighted original market capital to that of the current sum of the weighted market capital.

* Let ${X}$ = {${x_i | i \in I}$} the set of crypto currency assets; where $I \subset \mathbb{N}$ is an index set 
* respectively the set of asset market capitalization at time ${t}$: ${R}$(${t,X}$) = {${R}$(${t,x_i}$) | ${i \in I}$}
* the top ${N}$ market capitalization assets at time ${t}$: ${R}$(${t,{Y}}$) = {${R}$(${t,{x_{k_i}}}$) | ${k_i \in I_N}$}; where ${I_N}$ = {${k_i \in I}$}${_{i=1}^{N<n}}$
* The portfolio weight allocation for the top ${N<n}$ assets at time ${t}$: ${W}$(${t,Y}$) = {${w}$(${t,x_{k_i}}$) | ${k_i \in I_N}$}; such that $\sum_{k_i=1}^{N<n} {w_{t}(x_{k_{i}}) = 1.0}$
* Sum of the weighted portfolio allocation at time ${t}$: ${F_{t}({Y})}$  = $\sum_{k_i = 1}^{N<n} {w(t,x_{k_{i}})} \times {R(t,x_{k_i})}$
* __Objective__ is ${Max(F(t,Y))}$ > ${R^{'}(t)}$ by finding the optimal set of weights ${W(t,Y)}$; where ${R^{'}(t)}$ is an alternative ROI from a risk free investment

## Modern Portfolio Theorem
The [post-modern portfolio theory](https://www.investopedia.com/terms/p/pmpt.asp) (PMPT) attempts to improve on modern portfolio theory by minimizing downside risk instead of variance.

In [4]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('default')     #switch betweeb 'default' and 'ignore'

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

# Fund allocation and back testing

* [Estimate portfolio weights using Monte Carlo simulation](https://medium.com/analytics-vidhya/how-to-estimate-optimal-stock-portfolio-weights-using-monte-carlo-simulations-modern-portfolio-d27d534e8a1a)
* [Portfolio selection using Bayes Theorem](https://www.hindawi.com/journals/mpe/2019/4246903/)
* Introduction to [Modern Portfolio Theory](https://www.quantconnect.com/tutorials/introduction-to-financial-python/modern-portfolio-theory)
* [Practical Portfolio Optimisation](https://pythoninvest.com/long-read/practical-portfolio-optimisation)


1. Define the start and end date of the simulation cycle
   1. start_date: assumed date fund is allocated
   1. end_date: pseudo current date
1. Get the market capitalization data for entire time period
   1. Compute retrospective mean and standard deviation upto start_date
      1. compute for entire retrospective period
      1. compute for past 7, 20, and 100 days
   1. Estimate next day return based on simple returns formula
      1. estimate based on 7, 30, and 100 day intervals
      1. compare with the actual value of the following day
      1. estimate ratios based on USD & BTC as risk free returns
1. Get top N assets for each day to compute and continue from start-date to end-date
   1. fund allocation weights based on market capital expected returns
   1. Re-balance the weights after estimating the returns 

## Initiate dataETL class

In [210]:
import sys
sys.path.insert(1, '../lib')
import dataETL as etl

if debug:
    import importlib
    etl = importlib.reload(etl)

''' Initialize the dataETL class '''
path = "../data/market_cap_2021-01-01_2022-06-01/"
clsETL = etl.ExtractLoadTransform(dataPath=path)

## Load marketcap data

In [195]:
import pandas as pd

''' Get list of data file names '''
_l_fnames = clsETL.get_file_list()
print("Retrieved %d files in dir: %s " % (len(_l_fnames),path))
''' Load data into dataframe '''
rec_marketcap_df = clsETL.load_data(_l_fnames)
print("Loaded %d rows %s" % (rec_marketcap_df.shape[0],str(rec_marketcap_df.columns)))
''' Transform data with coin ids in columns '''
piv_marketcap_df = clsETL.transfrom_data(rec_marketcap_df, value_col_name='market_cap')
print("Transformed dataframe %d rows %s" % (piv_marketcap_df.shape[0],str(piv_marketcap_df.columns)))
print("Data load and transformation to pivot table complete!")


Retrieved 63 files in dir: ../data/market_cap_2021-01-01_2022-06-01/ 
Loaded 3637 rows Index(['Date', 'ID', 'Symbol', 'market_cap'], dtype='object')
Transformed dataframe 524 rows Index(['Date', 'bitcoin', 'cardano', 'ethereum', 'litecoin', 'ripple',
       'solana'],
      dtype='object')
Data load and transformation to pivot table complete!


## Filtered by date range

In [196]:
import datetime

start_dt = datetime.datetime(2022,1,1)
end_dt = datetime.datetime(2022,2,1)

''' >= start-date '''
if isinstance(start_dt, datetime.date):
    mask = (rec_marketcap_df["Date"] >= start_dt)
    rec_marketcap_df = rec_marketcap_df[mask]
    mask = (piv_marketcap_df["Date"] >= start_dt)
    piv_marketcap_df = piv_marketcap_df[mask]
''' <= end-date '''
if isinstance(end_dt, datetime.date):
    mask = (rec_marketcap_df["Date"] <= end_dt)
    rec_marketcap_df = rec_marketcap_df[mask]
    mask = (piv_marketcap_df["Date"] <= end_dt)
    piv_marketcap_df = piv_marketcap_df[mask]
print(rec_marketcap_df.info())
print(piv_marketcap_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224 entries, 0 to 31
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        224 non-null    datetime64[ns]
 1   ID          224 non-null    object        
 2   Symbol      224 non-null    object        
 3   market_cap  224 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 8.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 365 to 396
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      32 non-null     datetime64[ns]
 1   bitcoin   32 non-null     float64       
 2   cardano   32 non-null     float64       
 3   ethereum  32 non-null     float64       
 4   litecoin  32 non-null     float64       
 5   ripple    32 non-null     float64       
 6   solana    32 non-null     float64       
dtypes: dateti

## Top-N Assets

In [197]:
import numpy as np
topN = 3
_weights = np.array(clsETL.weights_matrix(N=topN,S=10)).reshape(-1,topN)
_topNassets_df = clsETL.get_topN_assets(rec_marketcap_df, N=topN)
_topNassets_df=_topNassets_df.reindex()

## Rolling mean and standard deviation

In [198]:
rolling_marketcap_mean = clsETL.rolling_mean(piv_marketcap_df, period=7)
transp_rolling_marketcap_mean = clsETL.transpose_pivot(rolling_marketcap_mean)
rolling_marketcap_stdv = clsETL.rolling_stdv(piv_marketcap_df, period=7)
transp_rolling_marketcap_stdv = clsETL.transpose_pivot(rolling_marketcap_stdv)

## Reshape dataframes to match TopN

In [199]:
matching_rolling_market_cap_mean = clsETL.match_dataframes(transp_rolling_marketcap_mean,_topNassets_df)
matching_rolling_market_cap_stdv = clsETL.match_dataframes(transp_rolling_marketcap_stdv,_topNassets_df)

## Instantiate ETP class

In [200]:
import sys
sys.path.insert(1, '../lib')
import assetETPreturns as returns

if debug:
    import importlib
    returns = importlib.reload(returns)

data_name = "coindecks"
clsROR = returns.RateOfReturns(name=data_name)
print(dir(clsROR))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__main__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'days_offset', 'get_geometric_return', 'get_holding_period_returns', 'get_logarithmic_returns', 'get_simple_returns', 'name', 'p_val', 'pd', 'sum_weighted_returns', 'window_length']


## Logarithmic returns
* return in period t can be expressed in terms of prices of period t and t + 1 in the following way ${R(t,x_{k_i}) = log \left({R(t,x_{k_i}) \over R(t+1,(x_{k_i})}\right)}$
* apply the logarithmic returns to actual market cap, the rolling mean, and rolling standard diviation 
* need to calculate the mean of daily logarithmic returns

In [212]:
import plotly.express as px

log_returns = clsROR.get_logarithmic_returns(_topNassets_df, value_col_name='market_cap')
trans_log_returns = clsETL.transfrom_data(log_returns,value_col_name="log")
_min_date = (trans_log_returns["Date"].min()).date()
_max_date = (trans_log_returns["Date"].max()).date()
_title = "Actuals logarithmic Returns from "+str(_min_date)+" to "+str(_max_date)
fig = px.scatter(trans_log_returns, x="Date", y=trans_log_returns.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title=_title)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

/home/gnewy/.local/lib/python3.8/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [214]:
mean_log_returns = clsROR.get_logarithmic_returns(
    matching_rolling_market_cap_mean.dropna(), 
    value_col_name='Value')
trans_mean_log_returns = clsETL.transfrom_data(mean_log_returns,value_col_name="log")
_min_date = (trans_mean_log_returns["Date"].min()).date()
_max_date = (trans_mean_log_returns["Date"].max()).date()
_title = "Mean logarithmic Returns from "+str(_min_date)+" to "+str(_max_date)
fig = px.scatter(trans_mean_log_returns, x="Date", y=trans_mean_log_returns.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title=_title)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

/home/gnewy/.local/lib/python3.8/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



## Sum of actual weighted returns

In [186]:
actual_returns = clsROR.sum_weighted_returns(_topNassets_df, _weights, value_col_name='market_cap')
actual_returns

[{'Date': Timestamp('2022-01-01 00:00:00'),
  'Expected Return': array([427089743511.52167, 403854270554.9484, 311437647878.94214,
         370345069009.45276, 253227196630.0674, 144089835729.1814,
         483540417373.71893, 579714423178.0613, 238469477584.73532,
         189684977303.14963], dtype=object)},
 {'Date': Timestamp('2022-01-02 00:00:00'),
  'Expected Return': array([437432258635.4762, 414412337768.2713, 319093960355.11035,
         381732405731.16833, 260547920416.12125, 149023519838.76166,
         496169956619.3926, 596511659512.7893, 245904271131.92896,
         196184580595.33112], dtype=object)},
 {'Date': Timestamp('2022-01-03 00:00:00'),
  'Expected Return': array([442843087418.2637, 417743978375.2305, 323071752599.0281,
         380835730920.25146, 261393434823.22568, 148165928583.39087,
         499965725105.4914, 596912096707.7501, 245483308426.9997,
         194689661750.46454], dtype=object)},
 {'Date': Timestamp('2022-01-04 00:00:00'),
  'Expected Return': a

In [187]:
log_weigthed_return = clsROR.sum_weighted_returns(mean_log_returns, _weights, value_col_name='Value')
log_weigthed_return

[{'Date': Timestamp('2022-01-07 00:00:00'),
  'Expected Return': array([426867355575.1831, 402864835079.70764, 311112764926.1051,
         367737082306.70465, 251846126351.00116, 142465934472.78818,
         482403523094.4433, 576751355729.5906, 236623807925.84937,
         187600477843.17694], dtype=object)},
 {'Date': Timestamp('2022-01-08 00:00:00'),
  'Expected Return': array([418950290887.1787, 395613005407.32324, 305127789308.53894,
         361634994544.3424, 247206653255.8161, 139704057089.124,
         473901351979.4474, 567352076786.5143, 232398530411.2644,
         184262858961.27686], dtype=object)},
 {'Date': Timestamp('2022-01-09 00:00:00'),
  'Expected Return': array([408251880258.86127, 385996235210.4828, 297160552968.2851,
         353944113094.41626, 241331470212.4739, 136510628113.94145,
         462558407275.64777, 555102697738.8893, 227194279674.47598,
         180350384375.85748], dtype=object)},
 {'Date': Timestamp('2022-01-10 00:00:00'),
  'Expected Return': arr

In [164]:
from datetime import date, timedelta

pred_marketcap_df = piv_marketcap_df.copy()
num_days = (end_dt - start_dt).days + 1
sharp = (_07_day_roll_simp_return_mean[_07_day_roll_simp_return_mean['Date'] == end_dt][_l_coin_ids] - 0.02/365)/ \
    _07_day_roll_simp_return_stdv[_07_day_roll_simp_return_stdv['Date'] == end_dt][_l_coin_ids]
#for now_date in (start_dt + timedelta(n) for n in range(num_days)):
#    print(now_date)
print(sharp)
print(simple_returns[simple_returns['Date'] == end_dt][_l_coin_ids])

      bitcoin   cardano  ethereum  litecoin   ripple    solana
334 -0.023394 -0.385817  0.184135 -0.077693 -0.19041 -0.145532
      bitcoin   cardano  ethereum  litecoin    ripple    solana
334 -0.014434 -0.033933  0.042531  0.003433  0.005133  0.019014


## Simple returns of Top N assets

In [116]:
import plotly.express as px

#transp_topNassets = clsETL.transpose_pivot(_topNassets_df)
#simple_returns_df = cls_etp.get_simple_returns(piv_marketcap_df)
simple_returns_df = clsROR.get_simple_returns(piv_marketcap_df)
_min_date = (simple_returns_df["Date"].min()).date()
_max_date = (simple_returns_df["Date"].max()).date()
_title = "Expected Returns from "+str(_min_date)+" to "+str(_max_date)
fig = px.scatter(simple_returns_df, x="Date", y=simple_returns_df.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title=_title)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

/home/gnewy/.local/lib/python3.8/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



## Holding Period Return

In [ ]:
hpr_df = clsReturns.get_holding_period_returns(piv_marketcap_df)
print("holding preriod return from \n%s to %s" % (str(start_dt.date()), str(end_dt.date())))
print("\n",hpr_df)